In [289]:
import numpy as np
import pandas as pd 
from numpy import nan
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import pairwise_distances
from correlation_pearson.code import CorrelationPearson
pearson = CorrelationPearson()
import operator

In [290]:
raw = pd.read_csv('WLO_raw(1).csv', delimiter=';')
# raw

In [291]:
raw = raw[pd.notnull(raw['rating'])]
raw = raw[pd.notnull(raw['timespent'])]
# raw

In [292]:
where_are_NaNs = isnan(raw)
raw[where_are_NaNs] = 0
# raw

In [293]:
E = raw['rating']
a = raw['bookmarked']
t = raw['timespent']
c = raw['total_selection']
b = np.exp(-t)

In [294]:
normalize = lambda x : ((x - np.min(x))/(np.max(x)-np.min(x)))
A = normalize(a)
B = normalize(b)
C = normalize(c)

In [295]:
e = E
i = A + ((2)*(B)) + ((2)*(C)*(E))
S = (1/2)*(e+i)

In [296]:
user_id = raw[['user_id']]
content_id = raw[['content_id']]
d = {
#     'user_id' : user_id,
#     'content_id' : content_id,
    'A' : A, 'B' : B, 'C' : C,
    'implicit' : i, 'explicit' : e, 'S' : S
}
data = pd.DataFrame(data = d)

# data

In [297]:
joinraw = raw.join(data)
weight = joinraw[['user_id', 'content_id', 'S']]
# weight

In [298]:
llor = weight.pivot_table(index='user_id', columns='content_id', values='S').fillna(0)
llor

content_id,32,33,34,35,36,37,43,44,45,46,...,52,53,54,55,56,57,58,59,60,61
user_id,,,,,,,,,,,,,,,,,,,,,
19,0.0,10.0,7.250000,5.500000,0.0,5.25,0.0,0.000000,5.5,0.000000,...,7.500000,5.0,10.049787,7.500000,5.00,0.000000,0.000000,7.500000,0.000000,10.000000
20,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,...,0.000000,3.5,0.000000,5.049787,5.00,5.018316,0.000000,0.000000,0.000000,0.000000
21,10.0,7.0,3.500000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,...,5.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,7.250000,8.750000,0.000000
22,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.00,13.000000,8.000000,7.500000,8.000000,8.000000
23,0.0,0.0,0.000000,0.000000,0.0,0.00,6.0,5.000335,0.0,5.000912,...,0.000000,0.0,0.000000,0.000000,7.50,0.000000,0.000000,0.000000,0.000000,0.000000
26,0.0,0.0,10.500912,5.500912,13.0,0.00,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,8.000000,0.00,8.002479,0.000000,10.500000,8.000000,8.000000
29,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,6.75,6.750000,0.000000,0.000000,13.500000,0.000000
30,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.00,7.500000,0.000000,0.000000,0.000000,4.500000
32,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,5.000000


In [299]:
llor_matrix = csr_matrix(llor.values)
llor_array = llor.values

In [300]:
dict_x={}
for i in range(len(llor_array)):
    dict_x[i]={}
    for j in range(len(llor_array)):
        if i==j:
            continue
        else:
            dict_x[i][j]= pearson.result(llor_array[i],llor_array[j])
    

In [304]:
dict_x={}
k=10
for i in range(len(llor_array)):
    print("=========INI USERID: ",i,"=================")
    dict_x[i]={}
    temp={}
    for j in range(len(llor_array)):
        if i==j:
            continue
        else:
            temp[j]= pearson.result(llor_array[i],llor_array[j])
    tmp = {k: temp[k] if not np.isnan(temp[k]) else 0 for k in temp}
#     dict_x[i] = dict(sorted(tmp.items(), key=operator.itemgetter(1),reverse=True)[:10])
    tmp = dict(sorted(tmp.items(), key=operator.itemgetter(1),reverse=True)[:k])
    pearsonDF = pd.DataFrame.from_dict(tmp, orient='index')
    pearsonDF.columns = ['similarityIndex']
    pearsonDF['user_id'] = pearsonDF.index
    pearsonDF.index = range(len(pearsonDF))
    mean_rating = [llor_array[y].mean() for y in list(pearsonDF['user_id'])]
    pearsonDF['ave_rating'] = mean_rating
#     print(pearsonDF)
    topUsersRating=pearsonDF.merge(weight, left_on='user_id', right_on='user_id', how='inner')
    topUsersRating['weight'] = topUsersRating['S'] - topUsersRating['ave_rating']
    topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['weight']
#     print(topUsersRating)
    tempTopUsersRating = topUsersRating.groupby('content_id').sum()[['similarityIndex','weightedRating']]
    tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
#     print(tempTopUsersRating)
    recommendation_df = pd.DataFrame()
    recommendation_df['weighted average recommendation score'] = llor_array[i].mean()+(tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex'])
    recommendation_df['content_id'] = tempTopUsersRating.index
#     print(recommendation_df)
    recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
    print(recommendation_df)
    

=========INI USERID:  0 =================
            weighted average recommendation score  content_id
content_id                                                   
57                                      11.223932          57
58                                      10.980425          58
60                                      10.980425          60
61                                      10.980425          61
59                                      10.480425          59
43                                       9.806405          43
47                                       9.674284          47
56                                       9.145831          56
46                                       8.807317          46
44                                       8.806740          44
55                                       7.420623          55
53                                       5.870836          53
=========INI USERID:  1 =================
            weighted average recommendation scor

            weighted average recommendation score  content_id
content_id                                                   
36                                      12.793478          36
34                                      10.294390          34
57                                       8.446125          57
59                                       8.152177          59
60                                       7.411103          60
61                                       7.411103          61
58                                       7.141304          58
55                                       5.756164          55
35                                       5.294390          35
49                                       5.293478          49
56                                       3.531715          56
53                                       2.031715          53
=========INI USERID:  14 =================
            weighted average recommendation score  content_id
content_id                 

=========INI USERID:  22 =================
            weighted average recommendation score  content_id
content_id                                                   
36                                      13.652174          36
54                                      10.157354          54
33                                      10.107567          33
59                                       9.626195          59
34                                       9.519087          34
61                                       9.278731          61
60                                       8.652174          60
47                                       8.107567          47
52                                       7.607567          52
55                                       7.187075          55
57                                       6.983436          57
49                                       6.132970          49
35                                       5.918236          35
45                         